In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from collections import defaultdict
import numpy as np
import pandas as pd
import anndata
from rnasieve.preprocessing import model_from_raw_counts

In [6]:
# Download example data. See example_data.ipynb to see how it was preprocessed.
#!pip3 install gdown
import gdown

!mkdir ..\example_data
gdown.download('https://drive.google.com/uc?id=1U3u-mbM7dnSSINqipq-fdjrMr8Fk__rr', '../example_data/muscle_subset_bulk.h5ad')
gdown.download('https://drive.google.com/uc?id=1wKQ_XWnK2tDJNS709u707uXlj2pqnvfX', '../example_data/muscle_subset.h5ad')

A subdirectory or file ..\example_data already exists.
Downloading...
From: https://drive.google.com/uc?id=1U3u-mbM7dnSSINqipq-fdjrMr8Fk__rr
To: C:\Users\nobre\OneDrive\Desktop\FALL 2021\BIO 500\Nobrega_Honors_Thesis\Nobrega_Honors_Thesis\example_data\muscle_subset_bulk.h5ad
100%|██████████| 4.01M/4.01M [00:00<00:00, 4.16MB/s]
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\nobre\\OneDrive\\Desktop\\FALL 2021\\BIO 500\\Nobrega_Honors_Thesis\\Nobrega_Honors_Thesis\\example_data\\muscle_subset_bulk.h5aduas5m7w3tmp'
Downloading...
From: https://drive.google.com/uc?id=1wKQ_XWnK2tDJNS709u707uXlj2pqnvfX
To: C:\Users\nobre\OneDrive\Desktop\FALL 2021\BIO 500\Nobrega_Honors_Thesis\Nobrega_Honors_Thesis\example_data\muscle_subset.h5ad
100%|██████████| 28.8M/28.8M [00:05<00:00, 5.27MB/s]
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\nobre\\OneDrive\\Desktop\\FALL 2021\\BIO 500\\No

In [7]:
# Example raw count prep
subset_bulk = anndata.read_h5ad('../example_data/muscle_subset_bulk.h5ad')
subset = anndata.read_h5ad('../example_data/muscle_subset.h5ad')

# Raw counts prep
print('Aggregating by ontology class...')
counts_by_onto_class = {}
for i in range(len(subset)):
    sc = subset[i]
    if len(sc.obs['cell_ontology_class']) == 0:
        continue
    cell_onto_class = sc.obs['cell_ontology_class'][0]
    if cell_onto_class not in counts_by_onto_class:
        counts_by_onto_class[cell_onto_class] = np.empty((sc.X.shape[1], 0), dtype=np.float32)
    counts_by_onto_class[cell_onto_class] = np.hstack(
        (counts_by_onto_class[cell_onto_class], sc.X.toarray().reshape(-1, 1)))
    
# Bulk prep
print('Aggregating bulks by age group...')
G = subset_bulk.n_vars
bulk_by_age = defaultdict(list)
for i in range(len(subset_bulk)):
    bulk = subset_bulk[i]
    if len(bulk.obs['Age']) == 0:
        continue
    age = bulk.obs['Age'][0]
    bulk_by_age[age].append(bulk.X.toarray().reshape(-1, 1))

bulk_labels = []
psis = np.empty((G, 0), dtype=np.float32)
for age in sorted(bulk_by_age.keys()):
    bulks = bulk_by_age[age]
    for i in range(len(bulks)):
        bulk_labels.append("{} months, subject {}".format(age, i))
        psis = np.hstack((psis, bulks[i]))
        
print('Done!')

Aggregating by ontology class...
Aggregating bulks by age group...
Done!


In [8]:
model, cleaned_psis = model_from_raw_counts(counts_by_onto_class, psis[:, :2]) #2 is arbitrary here

In [9]:
model.predict(cleaned_psis)

,B cell,T cell,endothelial cell,macrophage,mesenchymal stem cell,skeletal muscle satellite cell
Bulk 0,0.027962,0.0,0.149952,0.243373,0.146388,0.432325
Bulk 1,0.037864,0.0,0.141347,0.286524,0.219249,0.315016


In [10]:
# In this example, the intervals at a significance level of 0.05 indicate the estimate is poor.
# We set sig=0.9999 for the sake of visualization.
model.compute_marginal_confidence_intervals(sig=0.9999)

[[(0.005397159556887816, 0.05052618121334371),
  (0.0, 0.027483344629423374),
  (0.14701967720820378, 0.15288404127797853),
  (0.23995195912463785, 0.2467949809124842),
  (0.14294818334933937, 0.14982743272804674),
  (0.37960853252094834, 0.4850418521081296)],
 [(0.010698673728502063, 0.0650298248978245),
  (0.0, 0.02557179433682272),
  (0.13677379183665095, 0.14592113889683977),
  (0.28109788423676113, 0.2919491978683965),
  (0.2151222600765967, 0.2233753843716598),
  (0.25276135293683943, 0.37727049114992894)]]

In [11]:
model.plot_proportions('bar').properties(title="Muscle Proportion Estimates")

alt.FacetChart(...)

In [12]:
model.plot_proportions('stacked').properties(title="Muscle Proportion Estimates")

alt.Chart(...)